In [ ]:
BRANCH = 'ner_tutorial'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@{BRANCH}#egg=nemo_toolkit[nlp]

In [ ]:
from nemo.utils.exp_manager import exp_manager
from nemo.collections import nlp as nemo_nlp

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

# Task Description
**Named entity recognition (NER)**, also referred to as entity chunking, identification or extraction, is the task of detecting and classifying key information (entities) in text.
For example, in a sentence:  `Mary leaves in Santa Clara and works at NVIDIA.`, we should detect that `Mary` is a person, `Santa Clara` is location and `NVIDIA` is a company.

# Dataset

In this tutorial we going to use [GMB(Groningen Meaning Bank)](http://www.let.rug.nl/bjerva/gmb/about.php) corpus for entity recognition. 

GMB is a fairly large corpus with a lot of annotations. Note, that GMB is not completely human annotated and it’s not considered 100% correct. 
The data is labeled using the IOB format](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) (short for inside, outside, beginning). 

The following classes appear in the dataset:
* LOC = Geographical Entity
* ORG = Organization
* PER = Person
* GPE = Geopolitical Entity
* TIME = Time indicator
* ART = Artifact
* EVE = Event
* NAT = Natural Phenomenon

For this tutorial, classes ART, EVE, and NAT were combined into a MISC class due to small number of examples for these classes.




# NeMo Token Classification Data Format

[TokenClassification Model](https://github.com/NVIDIA/NeMo/blob/candidate/nemo/collections/nlp/models/token_classification/token_classification_model.py) in NeMo support NER and other token level classification tasks, as long as the data followes the format specified below. 

For NeMo Token Classification training scripts requires the data to be splitted into 2 files: 
* text.txt and 
* labels.txt. 

Each line of the **text.txt** file contains text sequences, where words are separated with spaces. The **labels.txt** file contains corresponding labels for each word in text.txt, the labels are separated with spaces. Each line of the files should follow the format: [WORD] [SPACE] [WORD] [SPACE] [WORD] (for text.txt) and [LABEL] [SPACE] [LABEL] [SPACE] [LABEL] (for labels.txt).

Example of a text.txt file:



```
Jennifer is from New York City .
She likes ...
...
```



Corresponding labels.txt file:



```
B-PER O O B-LOC I-LOC I-LOC O
O O ...
...
```



To convert an IOB format data to the format reqired for training, use this [script](https://github.com/NVIDIA/NeMo/blob/master/examples/nlp/token_classification/import_from_iob_format.py).

## Download and preprocess the data¶

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Let's extract files from the .zip file:

In [ ]:
DATA_DIR = 'for_tutorial'
data_zip = '/My\ Drive/Colab\ Notebooks/' + DATA_DIR + '.zip'
! unzip drive$data_zip

Now, the data folder should contain 4 files:



* labels_dev.txt
* labels_train.txt
* text_dev.txt
* text_train.txt


In [ ]:
! ls -l $DATA_DIR

In [ ]:
WORK_DIR = "PATH_TO_CHECKPOINTS_AND_LOGS"
MODEL_CONFIG = "token_classification_config.yaml"

In [ ]:
# let's take a look at the data 
print('Text:')
! head -n 5 {DATA_DIR}/text_train.txt

print('\nLabels:')
! head -n 5 {DATA_DIR}/labels_train.txt

# Model Configuration

Out Named Entity Recognition model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a Token Classification layer.

The model is defined in a config file which declares multiple important sections. They are:
- **model**: All arguments that are related to the Model - language model, token classifiers, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

In [ ]:
# download the model's configuration file 
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download('https://raw.githubusercontent.com/NVIDIA/NeMo/' + BRANCH + '/examples/nlp/token_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file is already exists')

In [ ]:
# this line will print the entire config of the model
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

# Setting up Data within the config

Among other things, the config file contains dictionaries called dataset, train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

We assume that both training and evaluation files are located in the same directory, and use the default names mentioned during the data download step. 
So, to start model training, simply specify `model.dataset.data_dir`, like we are going to do below.

Also notice that some configs, including `model.dataset.data_dir`, have `???` in place of paths, this values are required to be specified by the user.

Let's now add the data directory path to the config.

In [ ]:
# in this tutorial train and dev data is located in the same folder, so it is enought to add the path of the data directory to our config
config.model.dataset.data_dir = DATA_DIR

# Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!

Lets first instantiate a Trainer object!

In [ ]:
print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

In [ ]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
cuda = 1 if torch.cuda.is_available() else 0
config.trainer.gpus = cuda
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# For mixed precision training, use precision=16 and amp_level=O1

# Reduces maximum number of epochs to 1 for a quick training
config.trainer.max_epochs = 1

# Remove distributed training flags
config.trainer.distributed_backend = None

trainer = pl.Trainer(**config.trainer)

# Setting up a NeMo Experiment¶

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it!

In [ ]:
exp_dir = exp_manager(trainer, config.get("exp_manager", None))

# the exp_dir provides a path to the current experiment for easy access
exp_dir = str(exp_dir)
exp_dir

# Model Training

Before initializing the model, we might want to modify some of the model configs. For example, we might want to modify the pretrained BERT model.

In [ ]:
# complete list of supported BERT-like models
nemo_nlp.modules.get_pretrained_lm_models_list()

# specify BERT-like model, you want to use
PRETRAINED_BERT_MODEL = "bert-base-uncased"

In [ ]:
# add the specified above model parameters to the config
config.model.language_model.pretrained_model_name = PRETRAINED_BERT_MODEL

Now, we are ready to initialize our model. During the model initialization call, the dataset and data loaders we'll be prepared for training and evaluation.
Also, the pretrained BERT model will be downloaded, note it can take up to a few minutes depending on the size of the chosen BERT model.

In [ ]:
config.trainer.max_epochs = 10
model = nemo_nlp.models.TokenClassificationModel(cfg=config.model, trainer=trainer)

## Monitoring training progress
Optionally, you can create a Tensorboard visualization to monitor training progress.

In [ ]:
# load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir {exp_dir}

In [ ]:
# start model training
trainer.fit(model)

After training for 5 epochs, with the default config, your model performance should look similar to this: 
```
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         99.14      99.19      99.17     131141
    B-GPE (label_id: 1)                                     95.86      94.03      94.93       2362
    B-LOC (label_id: 2)                                     83.99      90.31      87.04       5346
    B-MISC (label_id: 3)                                    39.82      34.62      37.04        130
    B-ORG (label_id: 4)                                     78.33      67.82      72.70       2980
    B-PER (label_id: 5)                                     84.36      84.32      84.34       2577
    B-TIME (label_id: 6)                                    91.94      91.23      91.58       2975
    I-GPE (label_id: 7)                                     88.89      34.78      50.00         23
    I-LOC (label_id: 8)                                     77.18      79.13      78.14       1030
    I-MISC (label_id: 9)                                    28.57      24.00      26.09         75
    I-ORG (label_id: 10)                                    78.67      75.67      77.14       2384
    I-PER (label_id: 11)                                    86.69      90.17      88.40       2687
    I-TIME (label_id: 12)                                   83.21      83.48      83.34        938
    -------------------
    micro avg                                               96.95      96.95      96.95     154648
    macro avg                                               78.20      72.98      74.61     154648
    weighted avg                                            96.92      96.95      96.92     154648
```



# Inference

To see how the model performs, let’s run inference on a few examples.

In [ ]:
# define the list of queiries for inference
queries = [
    'we bought four shirts from the nvidia gear store in santa clara.',
    'Nvidia is a company.',
    'The Adventures of Tom Sawyer by Mark Twain is an 1876 novel about a young boy growing '
    + 'up along the Mississippi River.',
]
results = model.add_enties(queries)

for query, result in zip(queries, results):
    logging.info('')
    logging.info(f'Query : {query}')
    logging.info(f'Result: {result.strip()}\n')

We can also use our model to generate predictions for a dataset from a file, for example, for final evaluation or to perform error analysis. Below we are using the same dev set we were using for evaluation for training, but it could be any text file as long as it follows the data format described above.
Labels_file is optional here, and if provided will be used to get metrics and plot confusion matrix.

In [ ]:
# run evaluation on a dataset from file
model.evaluate_from_file(
    text_file=os.path.join(cfg.model.dataset.data_dir, cfg.model.validation_ds.text_file),
    labels_file=os.path.join(cfg.model.dataset.data_dir, cfg.model.validation_ds.labels_file),
    output_dir=exp_dir,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
)

# Training Script

If you have NeMo installed locally, you can also train the model with `nlp/token_classification/token_classification.py.`

To run training script, use:

`python token_classification.py model.dataset.data_dir=PATH_TO_DATA_DIR`